# CO-19 

## Using NLP to determine, connect Demand & Supply during COVID pandemic

### Main objective of this code was to analyze the data recorded from those who are willing to offer services during the pandemic and those who are willing to consume a perticular service but there isn't a dedicated platform in connecting the service providers with consumers

In [84]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://1.bp.blogspot.com/-aw6dxrzR44c/XVWjVZnANUI/AAAAAAAANwU/O8QFnX6lPawuEirvd1IVC-fGf9r2dkFCQCLcBGAs/s1600/cosine_tutorial.png")

In [85]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://www.machinelearningplus.com/wp-content/uploads/2018/10/soft-cosine.png")

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
demand = pd.read_excel('demand.xlsx')
supply = pd.read_excel('supply.xlsx')

### Using Tfidf Vectorizer 

In [3]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt') # if necessary...
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

[nltk_data] Downloading package punkt to /home/maltesh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

In [5]:
'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

In [6]:
vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

In [7]:
def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

### Overview of data

In [8]:
demand.head()

,demand,date,category,severity,like,location
0,sanitizers are empty so can someone help me kn...,2020-01-11,Grocery,High,5,JP NAGAR 5
1,are groceries running out in jp nagar 3th phase,2020-03-30,Grocery,Medium,8,JP NAGAR 3
2,where do I get fresh fruits and vegetables in ...,2020-03-20,Grocery,Medium,9,JP NAGAR
3,is it safe to buy and eat chicken,2020-01-03,Grocery,Low,3,JP NAGAR 1
4,where can I get rice bag in jp nagar 5th phasse,2020-01-05,Grocery,High,2,JP NAGAR 5


In [9]:
supply.head()

,supply,date,category,like,location
0,Free grocery supply delivery to home call 9845...,2020-03-24,Grocery,5,JP NAGAR 5
1,Fruits and vegetables avaiable at BTM Bank,2020-03-27,Grocery,8,BTM 2
2,Milk supply avaiable at RR Nagar Bus stop from...,2020-01-25,Grocery,9,RR Nagar
3,Hypermarket open at silkboard sector 1,2020-01-16,Grocery,3,SILK BOARD
4,Free food for needy between 1-4pm lunch,2020-03-25,Grocery,2,JP NAGAR 1


In [10]:
demand = pd.DataFrame(demand)
supply = pd.DataFrame(supply)

## User defined input asking the service needed along with location

In [11]:
my_demand = "does anyone know about grocery that can home deliver at jp nagar" 

In [12]:
my_location = 'JP NAGAR'

In [13]:
data = {}
senetnces = []
if my_demand:
    print("Sentence = ",my_demand)
    print()
    for i in range(0,len(supply.supply)):
        data[supply.supply[i]] = cosine_sim(my_demand,supply.supply[i])
import operator
sorted_d = dict(sorted(data.items(), key=operator.itemgetter(1),reverse=True))

Sentence =  does anyone know about grocery that can home deliver at jp nagar



### Cosine similarity with probability ranging from 0 - 1 
### 0 - Least Match
### 1 - Best Match

In [14]:
sorted_d

{'sanitizers and daily grocery avaiable at hyper store jp nagar ': 0.2329796548048752,
 'Free grocery supply delivery to home call 984589891': 0.1707761131901165,
 'Ups services avaiable at JP Nagar 5th phase': 0.1707761131901165,
 'Public food shelter avaiable at JP Nagar 7th phase near Yelchanaalli': 0.12735952979479354,
 'Free food delivery to home call 984589891': 0.08704446792504215,
 'Milk supply avaiable at RR Nagar Bus stop from 5-11am': 0.06744304769296745,
 'Mineral water service avaiable with home delivery of water cans': 0.06335481222196539,
 'Fruits and vegetables avaiable at BTM Bank': 0.0,
 'Hypermarket open at silkboard sector 1 ': 0.0,
 'Free food for needy between 1-4pm lunch': 0.0,
 'madiwala market open for fresh vegetables and fruits from 5-9am': 0.0,
 'Call this number for wifi connection 98745631': 0.0,
 'I can help those who want to learn AI': 0.0,
 'Xerox service avaible from 9-2pm': 0.0,
 'Ambulance services avaiable for free': 0.0,
 'Free masks and sanitizers

### Threshold of 0.1 i.e 10%, any service that matched the demand >10% is notified to user

In [15]:
threshold = 0.1
sentences = []

In [16]:
for i in sorted_d.items():
    if i[1] > threshold:
        sentences.append(i[0])

In [17]:
sentences

['sanitizers and daily grocery avaiable at hyper store jp nagar ',
 'Free grocery supply delivery to home call 984589891',
 'Ups services avaiable at JP Nagar 5th phase',
 'Public food shelter avaiable at JP Nagar 7th phase near Yelchanaalli']

In [18]:
s_rows = []
for i in range(len(supply)):
    if supply.iloc[i]['supply'] in sentences:
        s_rows.append(i)
    elif supply.iloc[i]['location'] in my_location:
        s_rows.append(i)

In [19]:
final_d = supply.iloc[s_rows]

In [20]:
final_d

,supply,date,category,like,location
0,Free grocery supply delivery to home call 9845...,2020-03-24,Grocery,5,JP NAGAR 5
5,sanitizers and daily grocery avaiable at hyper...,2020-03-30,Grocery,4,JP NAGAR 5
20,Ups services avaiable at JP Nagar 5th phase,2020-03-27,Other,0,JP NAGAR 5
21,Public food shelter avaiable at JP Nagar 7th p...,2020-03-10,Grocery,3,JP NAGAR 7


In [21]:
print("Here are the supplies that might help your demand.......")
print("Asked for = ",my_demand)
print()
print()
result = []
for i in range(len(final_d)):
    result.append({"Supply":final_d.iloc[i]['supply'],"Location":final_d.iloc[i]['location'],"Date Posted":final_d.iloc[i]['date']})
    print("Supply = ",final_d.iloc[i]['supply'])
    print("Location = ",final_d.iloc[i]['location'])
    print("Date Posted = ",final_d.iloc[i]['date'])
    print()
    print()

Here are the supplies that might help your demand.......
Asked for =  does anyone know about grocery that can home deliver at jp nagar


Supply =  Free grocery supply delivery to home call 984589891
Location =  JP NAGAR 5
Date Posted =  2020-03-24


Supply =  sanitizers and daily grocery avaiable at hyper store jp nagar 
Location =  JP NAGAR 5
Date Posted =  2020-03-30


Supply =  Ups services avaiable at JP Nagar 5th phase
Location =  JP NAGAR 5
Date Posted =  2020-03-27


Supply =  Public food shelter avaiable at JP Nagar 7th phase near Yelchanaalli
Location =  JP NAGAR 7
Date Posted =  2020-03-10




### This is the output that matches with the input demand and following services are shown to a user